# ResNet Example

In [3]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from pysnpe_utils import pysnpe
from pysnpe_utils.pysnpe_enums import *

import warnings
warnings.filterwarnings('ignore')


Enter path for SNPE_ROOT := 
 ./


ic| SNPE_ROOT: '.'
ic| ONNX_DIR: ('C:\\Users\\Pratik Sanghavi\\Desktop\\UW Madison\\Sem II\\CS744 Big Data '
               'Systems\\Project\\Code\\project_venv\\Lib\\site-packages\\onnx\\__init__.py')
ic| updated_pythonpath: '.\\lib\\python'
ic| updated_path: ('C:\\Users\\Pratik Sanghavi\\Desktop\\UW Madison\\Sem II\\CS744 Big Data '
                   'Systems\\Project\\Code\\project_venv\\Scripts;c:\\Users\\Pratik '
                   'Sanghavi\\.vscode\\extensions\\ms-python.python-2024.2.1\\pythonFiles\\deactivate\\powershell;C:\\Users\\Pratik '
                   'Sanghavi\\Desktop\\UW Madison\\Sem II\\CS744 Big Data '
                   'Systems\\Project\\Code\\project_venv\\Scripts;C:\\Users\\Pratik '
                   'Sanghavi\\miniconda3\\condabin;c:\\Users\\Pratik '
                   'Sanghavi\\.vscode\\extensions\\ms-python.python-2024.2.1\\pythonFiles\\deactivate\\powershell;C:\\Users\\Pratik '
                   'Sanghavi\\Desktop\\UW Madison\\Sem II\\CS744 Big Data '

TypeError: expected string or bytes-like object, got 'NoneType'

In [ ]:
device = torch.device("cpu")
print(f'Using {device} for inference')

In [ ]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

resnet50.eval().to(device)

In [ ]:
preprocessed_img = utils.prepare_input_from_uri('http://images.cocodataset.org/test-stuff2017/000000024309.jpg').to(device)

with torch.no_grad():
    torch_prediction = resnet50(preprocessed_img)

print(f'Predicted Class: {torch_prediction}')

In [ ]:
target_device = pysnpe.TargetDevice(target_device_type=DeviceType.X86_64_WINDOWS)

snpe_context = pysnpe.export_to_onnx(resnet50, [InputMap('img', (1, 3, 224, 224), torch.float32) ],
                                        ["out_class"], "resnet50.onnx")\
                                            .to_dlc()\
                                            .set_target_device(target_device=target_device)\
                                            .visualize_dlc()

input_tensor_map = {'img' : preprocessed_img.cpu().numpy()}
transpose_order = {'img' : (0,2,3,1)}    # ==> This is needed as SNPE needs input in NHWC (channel last format)

In [ ]:
# Run inference
snpe_out = snpe_context.execute_dlc(input_tensor_map, transpose_input_order=transpose_order)

In [ ]:
# Convert to Torch tensor format
snpe_prediction = torch.from_numpy(snpe_out['out_class'])